<a href="https://colab.research.google.com/github/rocketwolf98/ML_experiments/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
text = text[300000:800000]

In [ ]:
characters= sorted(set(text))

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [ ]:
sentences = []
next_characters = []

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i+SEQ_LENGTH])
  next_characters.append(text[i+SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype= bool)

for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1

Building Neural Network

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(128, input_shape = (SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size=256, epochs=4)
model.save('textgenerator.keras')

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 2.4769
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1.7606
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1.6061
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.5192


Text Generator

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [ ]:
print("for temperature 0.2")
print(generate_text(300, 0.2))
print("for temperature 0.4")
print(generate_text(300, 0.4))
print("for temperature 0.6")
print(generate_text(300, 0.6))
print("for temperature 0.8")
print(generate_text(300, 0.8))

for temperature 0.2
pardon of the prince, and call thee back.

king richard iii:
the can that honour of the casile shall not thine the sease
that the comes the brother of the love,
that he have not thy love in the land his canst the canst the sease.

king richard iii:
why, thou death that the fair death my father
that a grave that the sen the comes the sease
for temperature 0.4
f york:
nay, do not say, 'stand up;'
say not thy brother, that shall he should have the sease,
the warm, and can princh romeo and not shower.

king richard iii:
good more, that this grace, which son the consent
thou was thine the hand thy bears this land
thy death death, not that where i may thine so him.

king richard iii:
why, so i shal
for temperature 0.6
 edward iv:
welcome, sir john! but why confend use.

thomas of york:
but thou love, whose shall me worn for a grief,
this that i child stick proke hence to the want.

king richard iii:
hath than the warsick i praise thy son,
that i have sead thy nore that th